Import modules

In [1]:
import sys, yaml, logging, numpy as np
import torch, torch_geometric
if '/scratch' not in sys.path: sys.path.append('/scratch')
from GraphDUNE import datasets, models
from GraphDUNE.trainers.gnn_parallel import GNNParallelTrainer
from torch_geometric.data import DataListLoader

Configuration options

In [2]:
def configure(config):
  '''Load input configuration file'''
  with open(config) as f:
    return yaml.load(f, Loader=yaml.FullLoader)

Configuration options (overwrite default configuration with your own if you want!)

In [3]:
config = configure('/scratch/GraphDUNE/config/hit2d.yaml')

In [4]:
full_dataset = datasets.get_dataset(**config['data'])

device = torch.device(f'cuda:{config["model"]["gpus"][0]}' if torch.cuda.is_available() else 'cpu')

trainer = GNNParallelTrainer if len(config['model']['gpus']) > 1 else GNNTrainer
trainer = trainer(#real_weight=config['trainer']['real_weight'], fake_weight=config['trainer']['fake_weight'],
 output_dir='./test', device=device, summary_dir=config['trainer']['summary_dir'])

fulllen = len(full_dataset)
tv_num = np.ceil(fulllen*config['data']['t_v_split'])
splits = np.cumsum([fulllen-tv_num,0,tv_num])

Load dataset

In [5]:
train_dataset = torch.utils.data.Subset(full_dataset,np.arange(start=0,stop=splits[0]))
valid_dataset = torch.utils.data.Subset(full_dataset,np.arange(start=splits[1],stop=splits[2]))
# train_dataset = torch.utils.data.Subset(full_dataset,np.arange(start=0,stop=10*config['trainer']['batch_size']))
# valid_dataset = torch.utils.data.Subset(full_dataset,np.arange(start=10*config['trainer']['batch_size'],stop=20*config['trainer']['batch_size']))
loader = DataListLoader if len(config['model']['gpus']) > 1 else DataLoader
train_loader = loader(train_dataset, batch_size=config['trainer']['batch_size'], shuffle=True, pin_memory=True)
valid_loader = loader(valid_dataset, batch_size=config['trainer']['batch_size'], shuffle=False)

Build model

In [6]:
trainer.build_model(**config['model'])

Train!

In [7]:
train_summary = trainer.train(train_loader, config['trainer']['n_epochs'], valid_data_loader=valid_loader)
print(train_summary)

loss = 0.43501:  49%|████▉     | 3076/6266 [16:28<15:25,  3.45it/s]

RuntimeError: Caught RuntimeError in replica 1 on device 7.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "/scratch/GraphDUNE/models/message_passing.py", line 87, in forward
    return self.edge_network(data).squeeze(-1)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "/scratch/GraphDUNE/models/message_passing.py", line 32, in forward
    B = torch.cat([data.x[col],data.x[row]],dim=-1).detach()
RuntimeError: CUDA out of memory. Tried to allocate 1.01 GiB (GPU 7; 15.77 GiB total capacity; 12.04 GiB already allocated; 267.25 MiB free; 14.62 GiB reserved in total by PyTorch)
